In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torch import Generator
#from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import LambdaLR,StepLR
#from torch.utils.tensorboard import SummaryWriter

import config
from model import Model, Focalloss
from loadspec import load_data
from train_visual import train
from preprocess import RawDataset,SpecDataset, BucketSampler, train_preprocess, test_preprocess
seed=Generator()

/home/zhuyy/.conda/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zhuyy/.conda/envs/torch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
%%time
#train_data=load_data("data/cross_valid/cross.cat.mgf.train.repeat")
#val_data=load_data("data/cross_valid/cross.cat.mgf.valid.repeat")
train_data = load_data("data/human_high/peaks.db.mgf.train.unshared")
val_data = load_data("data/human_high/peaks.db.mgf.valid.unshared")

Total spectra: 54814 Valid spectra: 51790
Total spectra: 11665 Valid spectra: 11079
data loaded. Time: 0:00:05.248480


In [6]:
train_set = RawDataset(train_data)
val_set = RawDataset(val_data)

train_sampler = BucketSampler(data_source=train_set,
                              batch_size=config.BATCH_SIZE, buckets=config._buckets,generator=seed)
train_loader=DataLoader(train_set,batch_sampler=train_sampler,collate_fn=train_preprocess,num_workers=6)
val_sampler = BucketSampler(data_source=val_set,
                              batch_size=config.BATCH_SIZE, buckets=config._buckets,shuffle=False)
val_loader=DataLoader(val_set,batch_sampler=val_sampler,collate_fn=train_preprocess,num_workers=6)

padded length: 12	bucket size: 9851	batch count: 76
padded length: 17	bucket size: 20604	batch count: 160
padded length: 22	bucket size: 12971	batch count: 101
padded length: 32	bucket size: 8364	batch count: 65
total spectra: 51790	total batch: 402
padded length: 12	bucket size: 2223	batch count: 17
padded length: 17	bucket size: 4332	batch count: 33
padded length: 22	bucket size: 2656	batch count: 20
padded length: 32	bucket size: 1868	batch count: 14
total spectra: 11079	total batch: 84


In [4]:
train_model=Model(direction=2,lr=0.0005).to(device=config.device)
#checkpoint=torch.load('./ckpt/human_unshared_50000/default_backward.pth')
#train_model.load_state_dict(checkpoint["model"])
#train_model.opt.load_state_dict(checkpoint["optimizer"])
lr_scheduler=StepLR(train_model.opt,step_size=20,gamma=0.5)
#train_model=res_model.Model(direction=2,lr=0.001).to(device=config.device)

In [6]:
for name,param in train_model.named_parameters():
    if "backward" in name:
        param.requires_grad=True
    else:
        param.requires_grad=False

In [7]:
train_losses,val_losses=train(train_model,train_model.opt,train_loader,val_loader,
    n_iters=100,lr_scheduler=lr_scheduler,direction=2,
    save_path='./ckpt/human_unshared_50000/default_bidirection_cotrain.pth',
                              resume='./ckpt/human_unshared_50000/default_bidirection_cotrain.pth')

  0%|                                                                                                                    | 0/100 [00:00<?, ?it/s]/home/zhuyy/.conda/envs/torch/lib/python3.8/site-packages/torch/nn/modules/conv.py:453: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1660081234118/work/aten/src/ATen/native/Convolution.cpp:883.)
  return F.conv2d(input, weight, bias, self.stride,
  1%|█                                                                                                        | 1/100 [02:00<3:19:31, 120.93s/it]

epoch: 197	time: 0:01:57.524878
            train_loss: 2.0734	train_precision: 0.3756	train_recall: 0.3707
            val_loss: 2.3020	val_precision: 0.3093	val_recall:0.3919
            best_epoch: 196	loss: 2.3020	pre: 0.3093	rec: 0.3919


  2%|██                                                                                                       | 2/100 [04:02<3:18:14, 121.38s/it]

epoch: 198	time: 0:02:00.655615
            train_loss: 2.0724	train_precision: 0.3761	train_recall: 0.3714
            val_loss: 2.3027	val_precision: 0.3094	val_recall:0.3920
            best_epoch: 196	loss: 2.3020	pre: 0.3093	rec: 0.3919


  3%|███▏                                                                                                     | 3/100 [05:55<3:10:05, 117.58s/it]

...

epoch: 294	time: 0:02:01.844051
            train_loss: 2.0632	train_precision: 0.3802	train_recall: 0.3731
            val_loss: 2.2994	val_precision: 0.3106	val_recall:0.3926
            best_epoch: 249	loss: 2.2992	pre: 0.3104	rec: 0.3924


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [3:21:00<02:02, 122.64s/it]

epoch: 295	time: 0:02:01.289325
            train_loss: 2.0652	train_precision: 0.3799	train_recall: 0.3728
            val_loss: 2.2995	val_precision: 0.3106	val_recall:0.3925
            best_epoch: 249	loss: 2.2992	pre: 0.3104	rec: 0.3924


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [3:23:03<00:00, 121.83s/it]

epoch: 296	time: 0:02:00.491826
            train_loss: 2.0638	train_precision: 0.3802	train_recall: 0.3730
            val_loss: 2.2994	val_precision: 0.3106	val_recall:0.3926
            best_epoch: 249	loss: 2.2992	pre: 0.3104	rec: 0.3924


In [ ]:
with cProfile.Profile() as pr:
    train_loop(train_model,train_loader,direction=0)
pr.print_stats()

In [ ]:
for lay in train_model.modules():
    if isinstance(lay,torch.nn.LSTMCell):
        print(lay.weight_ih)